In [11]:
df = pd.read_csv("insurance_data.csv")

In [12]:
print(df.head())  # Display the first few rows
print(df.isnull().sum())  # Check for missing values
print(df.describe())  # Summary statistics

  Policy_ID  Customer_Age  Gender Region Policy_Type  Premium_Amount (USD)  \
0  POL_0001            76  Female  South        Home               1334.50   
1  POL_0002            72   Other  North      Health               1446.53   
2  POL_0003            71   Other  North      Travel               2103.51   
3  POL_0004            35   Other   West        Life               2451.76   
4  POL_0005            24    Male   East      Travel               1633.81   

   Claim_Amount (USD) Claim_Status  Number_of_Claims  Vehicle_Age  \
0                0.00      Pending                 0          NaN   
1            45619.84     Approved                 4          NaN   
2                0.00      Pending                 5          NaN   
3                0.00     Approved                 1          NaN   
4             9898.76     Approved                 1          NaN   

  Smoking_Status Medical_History  
0            NaN             NaN  
1         Smoker   Heart Disease  
2          

In [16]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("insurance_data.csv")

# Handle missing values
df["Vehicle_Age"] = df["Vehicle_Age"].fillna(0)  # Fill missing Vehicle_Age with 0
df["Smoking_Status"] = df["Smoking_Status"].fillna("Unknown")  # Fill missing Smoking_Status with "Unknown"
df["Medical_History"] = df["Medical_History"].fillna("Unknown")  # Fill missing Medical_History with "Unknown"

# Create Age Groups
df["Age_Group"] = pd.cut(
    df["Customer_Age"],
    bins=[0, 18, 35, 50, 100],
    labels=["0-18", "19-35", "36-50", "51+"]
)

# Policy Type Summary
policy_summary = df.groupby("Policy_Type").agg({
    "Premium_Amount (USD)": "mean",
    "Claim_Amount (USD)": "mean",
    "Number_of_Claims": "sum"
}).reset_index()

# Flatten Claim_Status Distribution
claim_status_distribution = df.groupby(["Policy_Type", "Claim_Status"]).size().unstack(fill_value=0)
claim_status_distribution = claim_status_distribution.div(claim_status_distribution.sum(axis=1), axis=0) * 100
claim_status_distribution = claim_status_distribution.reset_index()

# Merge Policy Summary with Claim Status Distribution
policy_summary = pd.merge(policy_summary, claim_status_distribution, on="Policy_Type")

# Regional Analysis
region_summary = df.groupby(["Region", "Policy_Type"]).agg({
    "Premium_Amount (USD)": "mean",
    "Claim_Amount (USD)": "mean",
    "Number_of_Claims": "sum"
}).reset_index()

# Risk Factor Analysis (Auto Policies)
auto_df = df[df["Policy_Type"] == "Auto"]
auto_risk_summary = auto_df.groupby("Vehicle_Age").agg({
    "Claim_Amount (USD)": "mean",
    "Number_of_Claims": "sum"
}).reset_index()

# Risk Factor Analysis (Health Policies)
health_df = df[df["Policy_Type"] == "Health"]
health_risk_summary = health_df.groupby("Smoking_Status").agg({
    "Claim_Amount (USD)": "mean",
    "Number_of_Claims": "sum"
}).reset_index()

# Save the cleaned and aggregated data for Power BI
df.to_csv("cleaned_insurance_data.csv", index=False)  # Cleaned raw data
policy_summary.to_csv("policy_summary.csv", index=False)  # Policy-level summary
region_summary.to_csv("region_summary.csv", index=False)  # Regional summary
auto_risk_summary.to_csv("auto_risk_summary.csv", index=False)  # Auto risk summary
health_risk_summary.to_csv("health_risk_summary.csv", index=False)  # Health risk summary